In [1]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from morning_price import MorningPrice
from betfair import Betfair

pd.set_option('display.max_columns', None)

In [2]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [3]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [4]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [5]:
aa = MorningPrice(start_time, end_time)
results = aa.markets(start_time, end_time, trading)

In [6]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.233380131 16:45 Warwick (WIN)
1.233380209 16:55 Newcastle (WIN)
1.233380216 17:30 Newcastle (WIN)
1.233380223 18:00 Newcastle (WIN)
1.233380230 18:30 Newcastle (WIN)
1.233380237 19:00 Newcastle (WIN)
1.233380244 19:30 Newcastle (WIN)


In [7]:
df1 = aa.market_runner(results)
df1

,market_id,market_start_time,venue,market_type,selection_id,horse_name
0,1.233380131,2024-09-24 16:45:00,Warwick,WIN,69174753,Georgias Charm
1,1.233380131,2024-09-24 16:45:00,Warwick,WIN,928425,Honour High
2,1.233380131,2024-09-24 16:45:00,Warwick,WIN,74168371,Mustamento
3,1.233380131,2024-09-24 16:45:00,Warwick,WIN,65436307,Lady Buttercup
4,1.233380131,2024-09-24 16:45:00,Warwick,WIN,71023391,County Royal
...,...,...,...,...,...,...
69,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,24292276,Spartan Fighter
70,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,13123197,Tathmeen
71,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,51819018,Rogue Thunder
72,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,7072840,Quanah


In [8]:
df1.dtypes

market_id                    object
market_start_time    datetime64[ns]
venue                        object
market_type                  object
selection_id                  int64
horse_name                   object
dtype: object

In [9]:
market_id_list = aa.market_id_list(df1)

In [10]:
market_books1 = aa.market_books(market_id_list, trading)

In [11]:
market_books1

[[<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>]]

# from here i want to automate getting the price 5 minutes before every race
then need to run through the appropiate model (turf, chase, hurdle) and place the bet

In [12]:
df2 = aa.morning_price(market_books1)

# join df1 and df2

In [13]:
df3 = aa.join(df1, df2)

In [14]:
df3.head(15)

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,last_price_traded,total_matched
0,1.233380131,2024-09-24 16:45:00+00:00,Warwick,WIN,69174753,Georgias Charm,ACTIVE,1.39,95437.65
1,1.233380131,2024-09-24 16:45:00+00:00,Warwick,WIN,928425,Honour High,ACTIVE,5.30,95437.65
2,1.233380131,2024-09-24 16:45:00+00:00,Warwick,WIN,74168371,Mustamento,ACTIVE,24.00,95437.65
3,1.233380131,2024-09-24 16:45:00+00:00,Warwick,WIN,65436307,Lady Buttercup,ACTIVE,44.00,95437.65
4,1.233380131,2024-09-24 16:45:00+00:00,Warwick,WIN,71023391,County Royal,ACTIVE,42.00,95437.65
5,1.233380131,2024-09-24 16:45:00+00:00,Warwick,WIN,74168370,Glen Affric,ACTIVE,320.00,95437.65
6,1.233380131,2024-09-24 16:45:00+00:00,Warwick,WIN,71731028,Pleniflora,REMOVED,NaN,95437.65
7,1.233380209,2024-09-24 16:55:00+00:00,Newcastle,WIN,73420632,Orange Sky,ACTIVE,2.26,47634.46
8,1.233380209,2024-09-24 16:55:00+00:00,Newcastle,WIN,69474943,Titian Blue,ACTIVE,3.20,47634.46
9,1.233380209,2024-09-24 16:55:00+00:00,Newcastle,WIN,73805109,Arkinthestars,ACTIVE,7.00,47634.46


In [15]:
df4 = pd.read_csv('../data/daily/2024_09_24.csv')
df4['Race Time'] = pd.to_datetime(df4['Race Time'], format='%d/%m/%Y %I:%M:%S %p')

df4.head(15)

,Race Time,Course,Distance (y),Race Type,Position,Horse,Weight (pounds),SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",Proform Speed Rating,Evening Price,Breakfast Price,Morning Price,Won P/L Before
0,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Another Beautiful,137,NaN,1,0,NaN,21.0,21.0,15.00,-8.00
1,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Beguiling,129,NaN,1,0,NaN,34.0,34.0,34.00,-5.00
2,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Dandy's Angel,130,NaN,1,0,NaN,8.0,9.0,7.00,-7.42
3,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Dr Rio,138,NaN,1,0,NaN,5.0,5.0,5.50,-26.88
4,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Etretat,137,NaN,1,0,NaN,5.5,5.5,8.00,-16.10
5,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Jez Bomb,131,NaN,1,0,NaN,4.0,4.0,4.33,-3.25
6,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Just Hiss,142,NaN,1,0,NaN,15.0,15.0,13.00,-23.75
7,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Love Yours,134,NaN,1,0,NaN,8.0,7.0,8.00,-8.00
8,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Olympicus,137,NaN,1,0,NaN,12.0,12.0,12.00,-3.00
9,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Panama City,134,NaN,1,0,NaN,26.0,26.0,26.00,5.33


## try joining

In [16]:
test_df4 = df4[df4['Course']== 'Beverley']
test_df4

,Race Time,Course,Distance (y),Race Type,Position,Horse,Weight (pounds),SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",Proform Speed Rating,Evening Price,Breakfast Price,Morning Price,Won P/L Before
0,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Another Beautiful,137,NaN,1,0,NaN,21.0,21.0,15.0,-8.00
1,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Beguiling,129,NaN,1,0,NaN,34.0,34.0,34.0,-5.00
2,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Dandy's Angel,130,NaN,1,0,NaN,8.0,9.0,7.0,-7.42
3,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Dr Rio,138,NaN,1,0,NaN,5.0,5.0,5.5,-26.88
4,2024-09-24 13:40:00,Beverley,2187,Turf,NaN,Etretat,137,NaN,1,0,NaN,5.5,5.5,8.0,-16.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2024-09-24 17:37:00,Beverley,1860,Turf,NaN,Ribble Radiant,134,NaN,1,0,NaN,17.0,17.0,15.0,7.00
149,2024-09-24 17:37:00,Beverley,1860,Turf,NaN,Straight A,136,NaN,1,0,NaN,9.0,9.0,11.0,-8.00
150,2024-09-24 17:37:00,Beverley,1860,Turf,NaN,Stratocracy,137,NaN,1,0,NaN,9.0,9.0,15.0,2.00
151,2024-09-24 17:37:00,Beverley,1860,Turf,NaN,Timely Chance,119,NaN,1,0,NaN,67.0,67.0,101.0,-7.00


In [18]:
test_df5 = df3[df3['venue']== 'Beverley']
test_df5

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,last_price_traded,total_matched
